In [1]:
import sys
sys.path.insert(1, '../architecture')
# sys.path.insert(2, '../dataLoaders')
# from ResidualBlock import *
# from UNetBlocks import *
from Prior import *
from Posterior import *

In [2]:
def kl_loss(priors, posteriors):

    klLoss = {}
    for level, (posterior, prior) in enumerate(zip(posteriors.items(), priors.items())):
        klLoss[level] = torch.mean(kl.kl_divergence(posterior[1], prior[1]), (1,2))
    return klLoss

In [3]:
input_test = torch.rand([50,3,256,256])
seg_label_test = torch.rand([50,20,256,256])
fri_label_test = torch.rand([50,1,256,256])

In [14]:
priorM = prior(num_samples = 10, num_classes = 20, LatentVarSize = 6)
posteriorM = posterior(num_samples = 10, num_classes = 20, LatentVarSize = 6)

optimizer = torch.optim.Adam([*priorM.parameters(), *posteriorM.parameters()], lr =  0.001)
criterion = nn.BCEWithLogitsLoss(size_average = True, reduction = None, reduce = True)
criterion = nn.CrossEntropyLoss(reduction='none', label_smoothing = 0.14)

In [11]:
posteriors = posteriorM(torch.cat([input_test,seg_label_test, fri_label_test], 1))
seg, priors, fric = priorM(input_test, posteriors)

In [12]:
gt_fric = torch.full_like(fric, 2.)
gt_seg = torch.full_like(seg, 2.)

In [21]:
loss_fric = criterion(fric, gt_fric)
loss_seg = criterion(seg, gt_seg)
loss_kl = torch.sum(torch.stack([i for i in kl_loss(priors, posteriors).values()]), 0)
# total_loss = loss_fric + loss_seg + loss_kl

In [8]:
total_loss.backward()
optimizer.step()

In [19]:
torch.mean(loss_seg,(1,2))

torch.Size([50])

In [22]:
loss_kl

tensor([297.9222, 318.4168, 329.5488, 298.7978, 287.6593, 353.2296, 308.7106,
        331.3213, 291.3873, 287.5399, 316.1758, 338.8154, 301.2698, 345.7997,
        308.7054, 325.0792, 302.1965, 320.3433, 308.8245, 306.5759, 345.0124,
        313.9180, 296.8038, 296.3813, 342.2180, 302.0573, 301.4528, 290.6595,
        278.6417, 288.8604, 303.8060, 313.5017, 301.7496, 321.9257, 327.3779,
        297.9522, 331.7773, 314.5306, 295.8223, 354.6790, 300.5458, 319.3062,
        298.6108, 295.9146, 298.4786, 300.4749, 301.6452, 311.1354, 273.2830,
        307.2227], grad_fn=<SumBackward1>)